In [42]:
import os

In [43]:
%pwd

'e:\\NLP\\Text-Summarizer-Project'

In [23]:
os.chdir("../")

In [26]:
%pwd

'e:\\NLP\\Text-Summarizer-Project'

In [44]:
os.chdir("Text-Summarizer-Project")

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'Text-Summarizer-Project'

In [8]:
%pwd

'e:\\NLP\\Text-Summarizer-Project'

In [45]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [47]:
from textSummarizer.constants import *

In [48]:
from textSummarizer.utils.common import read_yaml, create_directories

In [62]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = None,
        params_filepath = None
        ):
        
        if config_filepath is None:
          config_filepath = CONFIG_FILE_PATH

        if params_filepath is None:
          params_filepath = PARAMS_FILE_PATH

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
    

In [53]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [54]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers} ")

        else:
            logger.info(f"file already exists of size: {get_size(path(self.config.local_data_file ))}")




    def extract_zip_file(self):
        """ 
        zip_file_path:str
        Extracts the zip file into the data directory
        Function returns None

        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [70]:
try:
    config= ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()


except Exception as e:
    raise e

[2023-06-26 19:01:37,260:INFO:common:yaml file: config\config.yaml loaded successfully]
[2023-06-26 19:01:37,272:INFO:common:yaml file: params.yaml loaded successfully]
[2023-06-26 19:01:38,303:INFO:common:created directory at: artifacts]
[2023-06-26 19:01:38,437:INFO:common:created directory at: artifacts/data_ingestion]
[2023-06-26 19:02:04,236:INFO:698396313:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6AE6:10AA:158A41:2002B2:6499934F
Accept-Ranges: bytes
Date: Mon, 26 Jun 2023 13:32:00 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830046-CCU
X-Cache: MISS
X-Cache-Hits:

In [61]:
from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")
